##  Contents<a id='2.1_Contents'></a>
* [1 Data wrangling](#1_Data_wrangling)
  * [1.1 Contents](#1.1_Contents)
  * [1.2 Introduction](#1.2_Introduction)
  * [1.3 Objectives](#1.3_Objectives)
  * [1.4 Imports](#1.4_Imports)
  * [1.5 Load New York Housing Data](#1.5_Newyork_Housing_Data)
  * [1.6 Explore The Data](#1.6_Explore_The_Data)
    * [1.6.1 Clean the data](#1.6.1 Clean)
    * [1.6.2 Drop Rows With No Price Data](#1.6.2_Drop_Rows_With_No_Price_Data)
    * [1.6.3 Target Area](#1.6.3_Target_Area)
    * [1.6.4 Number Of Missing Values By Column](#1.6.4_Number_Of_Missing_Values_By_Column)
    * [1.6.5 Relabeling of Columns](#1.6.5_Relabel_Columns)
      * [1.6.5.1 Fixing Datatypes of Columns](#1.6.5.1_Fixing_Datatypes_of_columns)
      * [1.6.5.2 Reverse Geolocator](#1.6.5.2_Reverse_Geolocator)
    * [1.6.6 Distribution](#1.6.6_Distribution)
      * [1.6.6.1 Number of distinct neighborhoods](#1.6.6.1_Number_of_distinct_neighborhoods)
      * [1.6.6.2 Distribution Of Housing Price By Neighborhood](#1.6.6.2_Distribution_Of_Housing_Price_By_Neighborhood)
  * [1.7 Neighborhood Statistics](#1.7_Neighborhood_Statistics)
  * [1.8 Save data](#1.8_Save_data)
  * [1.9 Summary](#1.9_Summary)


## 1.2 Introduction<a id='1.2_Introduction and Problem'></a>

Capital Fortune is a midwest real estate company who is looking to invest in the ever-growing NYC market and have bought a significant amount of land in  Brooklyn NY which they have start to build many houses that will be finished 2024. These houses will be single-family homes with  They are seeking help from us to estimate the best pricing for their houses as well as what type of housing equates to the highest net-profit.

## 1.3 Objectives<a id='1.3_Objectives'></a>

Our objective will be to give the best house estimates for Capital Fortune houses which are comparable to other houses in the NYC area, as well as determine which house type yield the highest profit. I will be analyzing similar build houses as well as how those houses price are affected via location and build. In order to achieve our goal we will be using Zillow's Housing dataset which has records upto 01/20/2021

## 1.4 Imports<a id='1.4_Imports'></a>

In [1]:
#Lets get our imports
import pandas as pd
import seaborn as sns
import statistics as stat
import numpy as np
import matplotlib.pyplot as plt
import re
import requests
import matplotlib.ticker as tick

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from collections import Counter

## 1.5 Load New York Housing Data<a id='1.5_Newyork_Housing_Data'></a>

In [2]:
#Lets load up the raw data aka nyc_raw_data
nyc_rd = pd.read_csv('../data/newyork_housing.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,5,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,

In [3]:
#Lets see what we're working with.
nyc_rd.info()
#Looks like there are 75630 observations and 1507 columns, thats a lot of features we going to have to shrink that.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75630 entries, 0 to 75629
Columns: 1507 entries, address/city to zpid
dtypes: bool(13), float64(440), int64(2), object(1052)
memory usage: 863.0+ MB


In [4]:
#Let's take a peek at what our dataframe looks like
nyc_rd.head()
#Right off the bat we see a lot of missing data and some columns which are similar like address 

,address/city,address/community,address/neighborhood,address/state,address/streetAddress,address/subdivision,address/zipcode,bathrooms,bedrooms,currency,...,schools/2/link,schools/2/name,schools/2/rating,schools/2/size,schools/2/studentsPerTeacher,schools/2/totalCount,schools/2/type,url,yearBuilt,zpid
0,New York,NaN,NaN,NY,60 Terrace View Ave,NaN,10463.0,2.0,5.0,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/60-Terrace-...,1920.0,3.155405e+07
1,Bronx,NaN,NaN,NY,625 W 246th St,NaN,10471.0,8.0,8.0,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/625-W-246th...,1940.0,2.985412e+07
2,Bronx,NaN,NaN,NY,716 W 231st St,NaN,10463.0,3.0,4.0,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/716-W-231st...,1920.0,2.985186e+07
3,Bronx,NaN,NaN,NY,750 W 232nd St,NaN,10463.0,6.0,5.0,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/750-W-232nd...,1950.0,2.985186e+07
4,Bronx,NaN,NaN,NY,632 W 230th St,NaN,10463.0,6.0,5.0,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/632-W-230th...,2020.0,2.077107e+09


In [5]:
#I want to create a index or id column so when we remove columns its easy to merge on a column
nyc_rd['id'] = nyc_rd.index


In [6]:
nyc_rd['id']

0            0
1            1
2            2
3            3
4            4
         ...  
75625    75625
75626    75626
75627    75627
75628    75628
75629    75629
Name: id, Length: 75630, dtype: int64

### 1.6.1 Clean the Data<a id='1.6.1_Clean_the_Data'></a>

In [7]:
#Lets do a deeper dive into the columns
pd.set_option('display.max_columns', None)
nyc_rd_columns = nyc_rd.columns.tolist()
#We can see see there a lot redundant, irrelevant, and columns with missing data which we dont need lets drop them specifically 
#Lets deal with the irrelevant columns with photos first 

### 1.6.2 Dropping Columns<a id='1.6.2_Dropping_Column'></a>


In [8]:
nyc_rd = nyc_rd.drop(nyc_rd.filter(like='photos',axis=1).columns,axis=1)
nyc_rd_columns = nyc_rd.columns.tolist()
#Lets look at our new columns
nyc_rd_columns
#Still a lot of work to be done there is a lot of columns with price history do we really need that many?
#Lets investigate

['address/city',
 'address/community',
 'address/neighborhood',
 'address/state',
 'address/streetAddress',
 'address/subdivision',
 'address/zipcode',
 'bathrooms',
 'bedrooms',
 'currency',
 'dateposted',
 'description',
 'homeStatus',
 'latitude',
 'livingArea',
 'longitude',
 'price',
 'priceHistory',
 'priceHistory/0/attributeSource/infoString1',
 'priceHistory/0/attributeSource/infoString2',
 'priceHistory/0/attributeSource/infoString3',
 'priceHistory/0/buyerAgent',
 'priceHistory/0/buyerAgent/name',
 'priceHistory/0/buyerAgent/photo',
 'priceHistory/0/buyerAgent/photo/url',
 'priceHistory/0/buyerAgent/profileUrl',
 'priceHistory/0/event',
 'priceHistory/0/postingIsRental',
 'priceHistory/0/price',
 'priceHistory/0/priceChangeRate',
 'priceHistory/0/sellerAgent',
 'priceHistory/0/sellerAgent/name',
 'priceHistory/0/sellerAgent/photo',
 'priceHistory/0/sellerAgent/photo/url',
 'priceHistory/0/sellerAgent/profileUrl',
 'priceHistory/0/showCountyLink',
 'priceHistory/0/source',
 'p

In [9]:
nyc_rd.filter(like='priceHistory', axis=1)

,priceHistory,priceHistory/0/attributeSource/infoString1,priceHistory/0/attributeSource/infoString2,priceHistory/0/attributeSource/infoString3,priceHistory/0/buyerAgent,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/postingIsRental,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/showCountyLink,priceHistory/0/source,priceHistory/0/time,priceHistory/1/attributeSource/infoString1,priceHistory/1/attributeSource/infoString2,priceHistory/1/attributeSource/infoString3,priceHistory/1/buyerAgent,priceHistory/1/buyerAgent/name,priceHistory/1/buyerAgent/photo,priceHistory/1/buyerAgent/photo/url,priceHistory/1/buyerAgent/profileUrl,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/sellerAgent,priceHistory/1/sellerAgent/name,priceHistory/1/sellerAgent/photo,priceHistory/1/sellerAgent/photo/url,priceHistory/1/sellerAgent/profileUrl,priceHistory/1/showCountyLink,priceHistory/1/source,priceHistory/1/time,priceHistory/2/attributeSource/infoString1,priceHistory/2/attributeSource/infoString2,priceHistory/2/attributeSource/infoString3,priceHistory/2/buyerAgent,priceHistory/2/buyerAgent/name,priceHistory/2/buyerAgent/photo,priceHistory/2/buyerAgent/photo/url,priceHistory/2/buyerAgent/profileUrl,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/sellerAgent,priceHistory/2/sellerAgent/name,priceHistory/2/sellerAgent/photo,priceHistory/2/sellerAgent/photo/url,priceHistory/2/sellerAgent/profileUrl,priceHistory/2/showCountyLink,priceHistory/2/source,priceHistory/2/time,priceHistory/3/attributeSource/infoString1,priceHistory/3/attributeSource/infoString2,priceHistory/3/attributeSource/infoString3,priceHistory/3/buyerAgent,priceHistory/3/buyerAgent/name,priceHistory/3/buyerAgent/photo,priceHistory/3/buyerAgent/photo/url,priceHistory/3/buyerAgent/profileUrl,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/sellerAgent,priceHistory/3/sellerAgent/name,priceHistory/3/sellerAgent/photo,priceHistory/3/sellerAgent/photo/url,priceHistory/3/sellerAgent/profileUrl,priceHistory/3/showCountyLink,priceHistory/3/source,priceHistory/3/time,priceHistory/4/attributeSource/infoString1,priceHistory/4/attributeSource/infoString2,priceHistory/4/attributeSource/infoString3,priceHistory/4/buyerAgent,priceHistory/4/buyerAgent/name,priceHistory/4/buyerAgent/photo,priceHistory/4/buyerAgent/photo/url,priceHistory/4/buyerAgent/profileUrl,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/sellerAgent,priceHistory/4/sellerAgent/name,priceHistory/4/sellerAgent/photo,priceHistory/4/sellerAgent/photo/url,priceHistory/4/sellerAgent/profileUrl,priceHistory/4/showCountyLink,priceHistory/4/source,priceHistory/4/time,priceHistory/5/attributeSource/infoString1,priceHistory/5/attributeSource/infoString2,priceHistory/5/attributeSource/infoString3,priceHistory/5/buyerAgent,priceHistory/5/buyerAgent/name,priceHistory/5/buyerAgent/photo,priceHistory/5/buyerAgent/photo/url,priceHistory/5/buyerAgent/profileUrl,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/sellerAgent,priceHistory/5/sellerAgent/name,priceHistory/5/sellerAgent/photo,priceHistory/5/sellerAgent/photo/url,priceHistory/5/sellerAgent/profileUrl,priceHistory/5/showCountyLink,priceHistory/5/source,priceHistory/5/time,priceHistory/6/attributeSource/infoString1,priceHistory/6/attributeSource/infoString2,priceHistory/6/attributeSource/infoString3,priceHistory/6/buyerAgent,priceHistory/6/buyerAgent/n

In [10]:
nyc_rd['priceHistory'] == 'NaN'
#We can see the priceHistory column is useless it all missing data lets drop this column
#I am going to have to drop a lot of columns which serve no purpose similar to priceHistory 


0        False
1        False
2        False
3        False
4        False
         ...  
75625    False
75626    False
75627    False
75628    False
75629    False
Name: priceHistory, Length: 75630, dtype: bool

In [11]:
#Most of the columns are a history of price change but we won't be needing this for our problem so we will be ridding ourselfs
#of all columns with priceHistory
nyc_rd = nyc_rd.drop(nyc_rd.filter(like='priceHistory',axis=1).columns,axis=1)
nyc_rd_columns = nyc_rd.columns.tolist()
#Lets look at our new columns
nyc_rd_columns

['address/city',
 'address/community',
 'address/neighborhood',
 'address/state',
 'address/streetAddress',
 'address/subdivision',
 'address/zipcode',
 'bathrooms',
 'bedrooms',
 'currency',
 'dateposted',
 'description',
 'homeStatus',
 'latitude',
 'livingArea',
 'longitude',
 'price',
 'propertyTaxRate',
 'resoFactsStats/aboveGradeFinishedArea',
 'resoFactsStats/accessibilityFeatures',
 'resoFactsStats/accessibilityFeatures/0',
 'resoFactsStats/accessibilityFeatures/1',
 'resoFactsStats/additionalParcelsDescription',
 'resoFactsStats/appliances',
 'resoFactsStats/appliances/0',
 'resoFactsStats/appliances/1',
 'resoFactsStats/appliances/2',
 'resoFactsStats/appliances/3',
 'resoFactsStats/appliances/4',
 'resoFactsStats/appliances/5',
 'resoFactsStats/appliances/6',
 'resoFactsStats/appliances/7',
 'resoFactsStats/appliances/8',
 'resoFactsStats/appliances/9',
 'resoFactsStats/appliances/10',
 'resoFactsStats/appliances/11',
 'resoFactsStats/appliances/12',
 'resoFactsStats/applian

In [12]:
nyc_rd.filter(like='resoFact', axis=1)
#There are a lot of good information here like the appliances in the house, garage, parking, etc but unfortunately 
#there are too many missing values in a lot of this so I will be selecting columns that have a significant amount of data
#as well as ridding the appliciances columns as it might add value to a house it doesn't relate to CapitalFortune
#Housing as they will be sold empty

,resoFactsStats/aboveGradeFinishedArea,resoFactsStats/accessibilityFeatures,resoFactsStats/accessibilityFeatures/0,resoFactsStats/accessibilityFeatures/1,resoFactsStats/additionalParcelsDescription,resoFactsStats/appliances,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/appliances/3,resoFactsStats/appliances/4,resoFactsStats/appliances/5,resoFactsStats/appliances/6,resoFactsStats/appliances/7,resoFactsStats/appliances/8,resoFactsStats/appliances/9,resoFactsStats/appliances/10,resoFactsStats/appliances/11,resoFactsStats/appliances/12,resoFactsStats/appliances/13,resoFactsStats/appliances/14,resoFactsStats/architecturalStyle,resoFactsStats/associationAmenities,resoFactsStats/associationAmenities/0,resoFactsStats/associationAmenities/1,resoFactsStats/associationFee,resoFactsStats/associationFee2,resoFactsStats/associationFeeIncludes,resoFactsStats/associationFeeIncludes/0,resoFactsStats/associationFeeIncludes/1,resoFactsStats/associationFeeIncludes/2,resoFactsStats/associationFeeIncludes/3,resoFactsStats/associationFeeIncludes/4,resoFactsStats/associationFeeIncludes/5,resoFactsStats/associationFeeIncludes/6,resoFactsStats/associationFeeIncludes/7,resoFactsStats/associationFeeIncludes/8,resoFactsStats/associationFeeIncludes/9,resoFactsStats/associationFeeIncludes/10,resoFactsStats/associationFeeIncludes/11,resoFactsStats/associationName,resoFactsStats/associationName2,resoFactsStats/associationPhone,resoFactsStats/associationPhone2,resoFactsStats/atAGlanceFacts,resoFactsStats/atAGlanceFacts/0/factLabel,resoFactsStats/atAGlanceFacts/0/factValue,resoFactsStats/atAGlanceFacts/1/factLabel,resoFactsStats/atAGlanceFacts/1/factValue,resoFactsStats/atAGlanceFacts/2/factLabel,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factLabel,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factLabel,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factLabel,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/atAGlanceFacts/6/factLabel,resoFactsStats/atAGlanceFacts/6/factValue,resoFactsStats/atAGlanceFacts/7/factLabel,resoFactsStats/atAGlanceFacts/7/factValue,resoFactsStats/atAGlanceFacts/8/factLabel,resoFactsStats/atAGlanceFacts/8/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsPartial,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/belowGradeFinishedArea,resoFactsStats/builderModel,resoFactsStats/builderName,resoFactsStats/buildingArea,resoFactsStats/buildingAreaSource,resoFactsStats/buildingFeatures,resoFactsStats/buildingFeatures/0,resoFactsStats/buildingFeatures/1,resoFactsStats/buildingFeatures/2,resoFactsStats/buildingFeatures/3,resoFactsStats/buildingFeatures/4,resoFactsStats/buildingFeatures/5,resoFactsStats/buildingName,resoFactsStats/canRaiseHorses,resoFactsStats/carportSpaces,resoFactsStats/cityRegion,resoFactsStats/commonWalls,resoFactsStats/communityFeatures,resoFactsStats/communityFeatures/0,resoFactsStats/communityFeatures/1,resoFactsStats/communityFeatures/2,resoFactsStats/communityFeatures/3,resoFactsStats/communityFeatures/4,resoFactsStats/communityFeatures/5,resoFactsStats/constructionMaterials,resoFactsStats/constructionMaterials/0,resoFactsStats/constructionMaterials/1,resoFactsStats/constructionMaterials/2,resoFactsStats/constructionMaterials/3,resoFactsStats/constructionMaterials/4,resoFactsStats/constructionMaterials/5,resoFactsStats/cooling,resoFactsStats/cooling/0,resoFactsStats/cooling/1,resoFactsStats/cooling/2,resoFactsStats/cooling/3,resoFactsStats/coveredSpaces,resoFactsStats/developmentStatus,resoFactsStats/electric,resoFactsStats/electric/0,resoFactsStats/electric/1,resoFactsStats/electric/2,resoFactsStats/electric/3,resoFactsStats/electric/4,resoFactsStats/elementarySchool,resoFactsStats/elementarySchoolDistrict,resoFactsStats/entr

In [13]:
reso_drop_list = [] 
for i in nyc_rd_columns:
    if 'reso' in i:
        reso_drop_list.append(i)
        
len(reso_drop_list)

606

In [14]:
#After some tedious work on seeing which reso columns has reliable data as most were missing and irrelevant these 
#were the columns left that seem relevant but there will be further cleaning.
reso_list = ['resoFactsStats/atAGlanceFacts/0/factValue', 'resoFactsStats/atAGlanceFacts/1/factValue', 
            'resoFactsStats/atAGlanceFacts/3/factLabel','resoFactsStats/atAGlanceFacts/4/factLabel', 'resoFactsStats/atAGlanceFacts/4/factValue',
            'resoFactsStats/atAGlanceFacts/7/factLabel','resoFactsStats/atAGlanceFacts/7/factValue', 'resoFactsStats/basement', 'resoFactsStats/bathrooms', 'resoFactsStats/bathroomsFull', 
            'resoFactsStats/bathroomsHalf', 'resoFactsStats/bedrooms', 'resoFactsStats/cityRegion',
            'resoFactsStats/hasCooling', 'resoFactsStats/hasFireplace', 'resoFactsStats/hasHeating', 'resoFactsStats/hasSpa',
            'resoFactsStats/homeType', 'resoFactsStats/livingArea', 'resoFactsStats/lotSize', 'resoFactsStats/parkingFeatures/0',
            'resoFactsStats/parkingFeatures/1', 'resoFactsStats/stories', 'resoFactsStats/taxAnnualAmount', 'resoFactsStats/taxAssessedValue',
             'resoFactsStats/yearBuilt']


In [15]:

reso_drop = [x for x in reso_drop_list if x not in reso_list]

In [16]:
#Making a new dataframe of only reso_list so I can drop all reso columns and re-merge with the reso columns
#we want
#Drop all columns with reso
nyc_rd = nyc_rd.drop(reso_drop,axis=1)


In [17]:
#Lets look into the address/community and address/neighborhood on surface I see a alot of NaN
nyc_rd['address/community'].isnull().sum()
#Looks like they are all NaN so we can drop

75630

In [18]:
nyc_rd['address/neighborhood'].isnull().sum()
#Most observation is missing we can drop as we can use address/city plus zipcode to identify the area

74953

In [19]:
nyc_rd['address/zipcode'].isnull().sum
#We will keep this col

<bound method Series.sum of 0        False
1        False
2        False
3        False
4        False
         ...  
75625    False
75626    False
75627    False
75628    False
75629    False
Name: address/zipcode, Length: 75630, dtype: bool>

In [20]:
nyc_rd['address/subdivision'].isnull().sum()
#Drop

75628

In [21]:
nyc_rd['resoFactsStats/basement'].isnull().sum()
#A lot missing but important factor in price of houses lets keep this

48534

In [22]:
#Lets check if bathrooms and resoFacts/bathrooms are the same if so lets drop one
print((nyc_rd['bathrooms'] != nyc_rd['resoFactsStats/bathrooms']).sum())
#Hmm thats weird 20k of oberservation have different values lets see if there are NaN
print(nyc_rd['bathrooms'].isnull().sum())
print(nyc_rd['resoFactsStats/bathrooms'].isnull().sum())
#We can see that resoFactsbathrooms has less missing values than  bathrooms col so lets drop bathrooms

20298
19053
15312


In [23]:
#Now we need to check the same for bedrooms
print((nyc_rd['bedrooms'] != nyc_rd['resoFactsStats/bedrooms']).sum())
print(nyc_rd['bedrooms'].isnull().sum())
print(nyc_rd['resoFactsStats/bedrooms'].isnull().sum())
#Seems like bedrooms col has more missing 

19464
19464
16328


In [24]:
#Now we need to check the city/region
print((nyc_rd['address/city'] != nyc_rd['resoFactsStats/cityRegion']).sum())
print(nyc_rd['address/city'].isnull().sum())
print(nyc_rd['resoFactsStats/cityRegion'].isnull().sum())
#Seems like the discrepancies is probably from the way the string was typed so either one can be dropped

773
1
1


In [25]:
#resoFactsStats/homeType col and resoFactsStats/atAGlanceFacts/0/factValue seem to have same values lets check

print((nyc_rd['resoFactsStats/homeType'] != nyc_rd['resoFactsStats/atAGlanceFacts/0/factValue']).sum())
print(nyc_rd['resoFactsStats/homeType'].isnull().sum())
print(nyc_rd['resoFactsStats/atAGlanceFacts/0/factValue'].isnull().sum())
#They both tell the type of house it is and since hometype has more missing i will drop it

373
0
286


In [26]:
#Need to do the same for living area now
print((nyc_rd['livingArea'] != nyc_rd['resoFactsStats/livingArea']).sum())
print(nyc_rd['livingArea'].isnull().sum())
print(nyc_rd['resoFactsStats/livingArea'].isnull().sum())
# They all appear like they are not equal because reso usings sqft after its value whereas livingArea col does not
#regardless it seems resoLivingArea has less missing data so we keept it 

75630
9211
9182


In [27]:
#esoFactsStats/atAGlanceFacts/1/factValue and reso/yearBuilt seem to both have the year the house was built
print((nyc_rd['resoFactsStats/atAGlanceFacts/1/factValue'] != nyc_rd['resoFactsStats/yearBuilt']).sum())
print(nyc_rd['resoFactsStats/atAGlanceFacts/1/factValue'].isnull().sum())
print(nyc_rd['resoFactsStats/yearBuilt'].isnull().sum())
print(nyc_rd['yearBuilt'].isnull().sum())
#Here there are 3 columns with the same info lets drop the two with the most missing values
#The biggest different is probably due to the fact one col has int and the other floats so lets drop the one with more missing values

44265
5688
5692
5732


In [28]:
#Further columns to drop
drop_cols = ['zpid', 'url', 'address/community', 'address/neighborhood', 'address/subdivision', 'currency', 'dateposted'
            , 'homeStatus', 'description', 'bathrooms', 'bedrooms', 'resoFactsStats/cityRegion', 'resoFactsStats/homeType'
            , 'resoFactsStats/atAGlanceFacts/3/factLabel', 'livingArea', 'resoFactsStats/yearBuilt', 'yearBuilt']

In [29]:
nyc_rd = nyc_rd.drop(drop_cols, axis=1)

In [30]:
#Lets look at our new DF we still need to clean out the schools columns
nyc_rd.head()

,address/city,address/state,address/streetAddress,address/zipcode,latitude,longitude,price,propertyTaxRate,resoFactsStats/atAGlanceFacts/0/factValue,resoFactsStats/atAGlanceFacts/1/factValue,resoFactsStats/atAGlanceFacts/4/factLabel,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/7/factLabel,resoFactsStats/atAGlanceFacts/7/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bedrooms,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasHeating,resoFactsStats/hasSpa,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/parkingFeatures/0,resoFactsStats/parkingFeatures/1,resoFactsStats/stories,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,schools,schools/0/assigned,schools/0/distance,schools/0/grades,schools/0/isAssigned,schools/0/level,schools/0/link,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/0/type,schools/1/assigned,schools/1/distance,schools/1/grades,schools/1/isAssigned,schools/1/level,schools/1/link,schools/1/name,schools/1/rating,schools/1/size,schools/1/studentsPerTeacher,schools/1/totalCount,schools/1/type,schools/2/assigned,schools/2/distance,schools/2/grades,schools/2/isAssigned,schools/2/level,schools/2/link,schools/2/name,schools/2/rating,schools/2/size,schools/2/studentsPerTeacher,schools/2/totalCount,schools/2/type,id
0,New York,NY,60 Terrace View Ave,10463.0,40.877743,-73.910866,799999.0,0.88,Residential,1920,Parking,Driveway,Price/sqft,$424,Finished,2.0,1.0,1.0,5.0,True,NaN,True,False,"1,889 sqft",NaN,Driveway,NaN,NaN,5096.0,711000.0,NaN,NaN,0.1,K-8,True,Elementary,https://www.greatschools.org/school?id=01885&s...,Ps 37 Multiple Intelligence School,4.0,647.0,14.0,1.0,Public,NaN,0.1,6-12,True,Middle,https://www.greatschools.org/school?id=06426&s...,In Tech Academy Aka Ms High School 368,3.0,993.0,14.0,1.0,Public,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Bronx,NY,625 W 246th St,10471.0,40.892689,-73.910667,3995000.0,0.95,Single Family,1940,Parking,"Garage, Garage - Attached",Price/sqft,$571,NaN,8.0,7.0,1.0,8.0,True,True,False,False,"7,000 sqft",0.29 Acres,Garage,Garage - Attached,1.0,13941.0,1937000.0,NaN,NaN,0.4,K-5,True,Elementary,https://www.greatschools.org/school?id=02100&s...,Ps 24 Spuyten Duyvil,10.0,907.0,16.0,1.0,Public,NaN,0.3,6-12,True,Middle,https://www.greatschools.org/school?id=08367&s...,Riverdale Kingsbridge Academy (Ms High School ...,5.0,1516.0,15.0,1.0,Public,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Bronx,NY,716 W 231st St,10463.0,40.883419,-73.918106,1495000.0,0.95,Single Family,1920,Parking,"Garage, Garage - Attached",Price/sqft,$353,NaN,3.0,3.0,0.0,4.0,False,NaN,False,False,"4,233 sqft",0.42 Acres,Garage,Garage - Attached,2.0,12253.0,2341000.0,NaN,NaN,0.3,K-5,True,Elementary,https://www.greatschools.org/school?id=02100&s...,Ps 24 Spuyten Duyvil,10.0,907.0,16.0,1.0,Public,NaN,0.4,6-12,True,Middle,https://www.greatschools.org/school?id=08367&s...,Riverdale Kingsbridge Academy (Ms High School ...,5.0,1516.0,15.0,1.0,Public,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,Bronx,NY,750 W 232nd St,10463.0,40.885033,-73.917793,3450000.0,0.95,Single Family,1950,Parking,"Garage, Garage - Attached",Price/sqft,$493,NaN,6.0,6.0,0.0,5.0,True,True,False,False,"7,000 sqft",0.26 Acres,Garage,Garage - Attached,2.0,19472.0,3011000.0,NaN,NaN,0.2,K-5,True,Elementary,https://www.greatschools.org/school?id=02100&s...,Ps 24 Spuyten Duyvil,10.0,907.0,16.0,1.0,Public,NaN,0.3,6-12,True,Middle,https://www.greatschools.org/school?id=08367&s...,Riverdale Kingsbridge Academy (Ms High School ...,5.0,1516.0,15.0,1.0,Public,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,Bronx,NY,632 W 230th St,10463.0,40.881702,-73.914185,1790000.0,0.95,Single Family,2020,Parking,0 spaces,NaN,NaN,NaN,6.0,5.0,1.0,5.0,True,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,K-5,True,Elementary,https://www

In [31]:
#On the surface i see a alot of NaN for schools lets check
nyc_rd['schools'].isnull().sum()
#Useless col we can drop


75630

In [32]:
#It seems our dataset gives us schools at each level elementary, middle and high schools
school_drop = ['schools', 'schools/0/assigned', 'schools/0/link', 'schools/0/totalCount', 'schools/1/assigned'
              , 'schools/1/link' ]
#Also all columns with schools/2 are NaN so lets drop using a filter

In [33]:
nyc_rd = nyc_rd.drop(nyc_rd.filter(like='schools/2',axis=1).columns,axis=1)

In [34]:
nyc_rd = nyc_rd.drop(school_drop,axis=1)

### 1.6.3 Target Area<a id='1.6.3_Target_Area'></a>

Capital Fortune seems to only concern about prices in Brooklyn while other boroughs have good information they might give us noise when it comes to predicting prices only in Brooklyn as areas in Manhattan is notoriously expensive where as state island is affordable. 

In [36]:
#Lets see the distribution of our observations and where they are located
Counter(nyc_rd['address/city'].to_list()).most_common()
#Staten Island seems have a lot of observations but we can see some of the city names are actually neighborhoods
#in their respected neighborhood
#Perticularly Queens 

[('Staten Island', 15735),
 ('Brooklyn', 14936),
 ('Bronx', 13706),
 ('Jamaica', 7566),
 ('Flushing', 6797),
 ('New York', 4342),
 ('Far Rockaway', 2211),
 ('Long Island City', 1011),
 ('Queens', 569),
 ('Little Neck', 491),
 ('Forest Hills', 446),
 ('Astoria', 360),
 ('Queens Village', 329),
 ('Howard Beach', 321),
 ('Bayside', 295),
 ('Fresh Meadows', 279),
 ('Glen Oaks', 276),
 ('Rosedale', 263),
 ('Woodside', 254),
 ('Glendale', 248),
 ('Oakland Gardens', 218),
 ('Laurelton', 213),
 ('Ridgewood', 211),
 ('Staten island', 209),
 ('East Elmhurst', 205),
 ('Breezy Point', 190),
 ('Jackson Heights', 189),
 ('Broad Channel', 178),
 ('Bellerose', 169),
 ('Rego Park', 159),
 ('Richmond Hill', 152),
 ('New york', 149),
 ('Hollis', 145),
 ('Rockaway Park', 142),
 ('Elmhurst', 142),
 ('Springfield Gardens', 126),
 ('South Richmond Hill', 113),
 ('Rockaway Beach', 112),
 ('College Pt', 112),
 ('Maspeth', 107),
 ('Whitestone', 107),
 ('Corona', 102),
 ('Kew Gardens', 91),
 ('Douglaston', 85),


In [37]:
#Lets seperate our DataFrame for brooklyn
brooklyn_df = nyc_rd[nyc_rd['address/city'] == 'Brooklyn']

### 1.6.4 Number Of Missing Values By Column<a id='1.6.4_Number_Of_Missing_Values_By_Column'></a>

In [38]:
#Okay we refine our dataset by a lot but there is still alot more work, we need to relabel and perphaps drop a few more cols
#This isn't perfect as still a lot of columns have NaN but its a start we will do further work as we work towards our objective.
#Lets see our missing data 
missing = pd.concat([brooklyn_df.isnull().sum(), 100 * brooklyn_df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count', '%'])
#Lets start with dropping columns that have a lot of NaN and relabeling columns to better represent the column values
#And for some of the more trivial one where its NaN we will replace with either 0 or false
#For example has fireplace if its NaN we will just put false or 
#how many stories a build it is if its NaN we can assume its 0 

,count,%
address/city,0,0.000000
address/state,0,0.000000
address/streetAddress,0,0.000000
resoFactsStats/hasSpa,0,0.000000
id,0,0.000000
propertyTaxRate,1,0.006695
latitude,3,0.020086
longitude,3,0.020086
address/zipcode,4,0.026781
price,6,0.040171


### 1.6.5 Relabeling of Columns<a id='1.6.4_Relabel_Columns'></a>

In [39]:
#Some of the remaining columns are labels for another column so we will drop that and rename those column to 
# the right labels
drop_cols = ['resoFactsStats/atAGlanceFacts/4/factLabel', 'resoFactsStats/atAGlanceFacts/7/factLabel',
            'resoFactsStats/parkingFeatures/0', 'resoFactsStats/parkingFeatures/1', 'schools/1/totalCount',
            'resoFactsStats/atAGlanceFacts/7/factValue',]

column_rename = {'address/city':'City', 'address/state': 'State', 'address/zipcode':'ZipCode', 
                 'resoFactsStats/atAGlanceFacts/0/factValue': 'HomeType', 
                 'resoFactsStats/atAGlanceFacts/1/factValue':'YearBuilt', 
                 'resoFactsStats/atAGlanceFacts/4/factValue': 'ParkingType',
                'resoFactsStats/basement':'Basement', 'resoFactsStats/bathrooms': 'Total_Bathrooms',
                 'resoFactsStats/bathroomsFull':'Full_Bathrooms','resoFactsStats/bathroomsHalf':'Half_Bathrooms',
                 'resoFactsStats/bedrooms':'Bedrooms', 'resoFactsStats/hasCooling':'Cooling_Available',
                 'resoFactsStats/hasFireplace': 'Fireplace_Available', 'resoFactsStats/hasHeating': 'Heating_Available',
                 'resoFactsStats/hasSpa':'Spa_Available', 'resoFactsStats/livingArea': 'LivingArea_Space(SqFT)',
                 'resoFactsStats/lotSize':'Lot_Size', 'resoFactsStats/stories':'Story_Building',
                 'resoFactsStats/taxAnnualAmount':'Annual_Tax', 'resoFactsStats/taxAssessedValue':'Tax_Assessed_Value'}

In [40]:
brooklyn_df = brooklyn_df.drop(drop_cols, axis=1)

In [41]:
brooklyn_df.rename(columns = column_rename, inplace = True)

In [42]:
brooklyn_df.head()
##Nice now that we have relabed properly lets drop rest of those columns

,City,State,address/streetAddress,ZipCode,latitude,longitude,price,propertyTaxRate,HomeType,YearBuilt,ParkingType,Basement,Total_Bathrooms,Full_Bathrooms,Half_Bathrooms,Bedrooms,Cooling_Available,Fireplace_Available,Heating_Available,Spa_Available,LivingArea_Space(SqFT),Lot_Size,Story_Building,Annual_Tax,Tax_Assessed_Value,schools/0/distance,schools/0/grades,schools/0/isAssigned,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/type,schools/1/distance,schools/1/grades,schools/1/isAssigned,schools/1/level,schools/1/name,schools/1/rating,schools/1/size,schools/1/studentsPerTeacher,schools/1/type,id
1389,Brooklyn,NY,177 Monitor St,11222.0,40.725403,-73.942284,1385000.0,0.65,Single Family,1899,0 spaces,NaN,2.0,2.0,0.0,3.0,True,True,True,False,"1,425 sqft","1,200 sqft",2.0,3496.0,1514000.0,0.1,PK-5,True,Primary,Ps 110 The Monitor,4.0,522.0,14.0,Public,0.5,6-8,True,Middle,John Ericsson Middle School 126,4.0,377.0,11.0,Public,1389
1391,Brooklyn,NY,239 Monitor St,11222.0,40.727341,-73.943130,1399000.0,0.65,Single Family,1901,0 spaces,Finished,3.0,2.0,1.0,3.0,True,False,True,False,"1,800 sqft","1,123 sqft",NaN,3488.0,1168000.0,0.2,PK-5,True,Primary,Ps 110 The Monitor,4.0,522.0,14.0,Public,0.6,6-8,True,Middle,John Ericsson Middle School 126,4.0,377.0,11.0,Public,1391
2405,Brooklyn,NY,71 Wolcott St APT 206A,11231.0,40.678719,-74.014549,799000.0,0.65,Condo,2019,0 spaces,NaN,2.0,1.0,1.0,3.0,False,NaN,False,False,NaN,NaN,4.0,NaN,NaN,0.2,PK-5,True,Primary,Ps 15 Patrick F Daly,2.0,487.0,9.0,Public,0.8,6-12,True,Middle,Brooklyn Secondary School For Collaborative St...,3.0,675.0,11.0,Public,2405
2406,Brooklyn,NY,161 Coffey St,11231.0,40.677280,-74.016098,3600000.0,0.65,Single Family,1899,0 spaces,NaN,4.0,2.0,2.0,3.0,True,True,False,False,"1,536 sqft","1,667 sqft",2.0,2433.0,1895000.0,0.3,PK-5,True,Primary,Ps 15 Patrick F Daly,2.0,487.0,9.0,Public,0.9,6-12,True,Middle,Brooklyn Secondary School For Collaborative St...,3.0,675.0,11.0,Public,2406
2407,Brooklyn,NY,434 Van Brunt St,11231.0,40.675514,-74.017532,2649500.0,0.65,Townhouse,2019,0 spaces,See Remarks,5.0,3.0,2.0,4.0,True,True,False,False,"4,082 sqft",NaN,NaN,NaN,NaN,0.4,PK-5,True,Primary,Ps 15 Patrick F Daly,2.0,487.0,9.0,Public,1.0,6-12,True,Middle,Brooklyn Secondary School For Collaborative St...,3.0,675.0,11.0,Public,2407


In [43]:
#Lets check out all rows with price less than 50k
brooklyn_df.loc[nyc_rd['price'] < 50000]

#Seems to me these prices are not accurate and more like family members selling each other their property to transfer title
#For sake of simplicity I will drop these rows and our client houses being built are all above 100k

#We will also drop all houses over 100Million as most of these are warehouses and outside of Capital Fortune houses price

,City,State,address/streetAddress,ZipCode,latitude,longitude,price,propertyTaxRate,HomeType,YearBuilt,ParkingType,Basement,Total_Bathrooms,Full_Bathrooms,Half_Bathrooms,Bedrooms,Cooling_Available,Fireplace_Available,Heating_Available,Spa_Available,LivingArea_Space(SqFT),Lot_Size,Story_Building,Annual_Tax,Tax_Assessed_Value,schools/0/distance,schools/0/grades,schools/0/isAssigned,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/type,schools/1/distance,schools/1/grades,schools/1/isAssigned,schools/1/level,schools/1/name,schools/1/rating,schools/1/size,schools/1/studentsPerTeacher,schools/1/type,id
2430,Brooklyn,NY,130 Van Dyke St,11231.0,40.676102,-74.014923,10.0,0.65,Multiple Occupancy,1901,0 spaces,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,"1,650 sqft","2,100 sqft",2.0,2730.0,1600000.0,0.2,PK-5,True,Primary,Ps 15 Patrick F Daly,2.0,487.0,9.0,Public,0.8,6-12,True,Middle,Brooklyn Secondary School For Collaborative St...,3.0,675.0,11.0,Public,2430
10367,Brooklyn,NY,One Blue Slip #28B,11222.0,40.732983,-73.962425,6970.0,0.88,Apartment,NaN,0 spaces,NaN,2.0,2.0,0.0,2.0,False,NaN,False,False,"1,100 sqft",NaN,NaN,NaN,NaN,0.7,PK-5,True,Primary,Ps 31 Samuel F Dupont,8.0,594.0,15.0,Public,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10367
10394,Brooklyn,NY,149 Huron St APT 1C,11222.0,40.733078,-73.955711,3600.0,0.65,Available Now,Multiple Occupancy,"Cats, Small Dogs, Dogs",NaN,NaN,0.0,0.0,2.0,True,NaN,True,False,"1,053 sqft",NaN,6.0,NaN,NaN,0.4,PK-5,True,Primary,Ps 31 Samuel F Dupont,8.0,594.0,15.0,Public,1.0,6-8,True,Middle,John Ericsson Middle School 126,4.0,377.0,11.0,Public,10394
10647,Brooklyn,NY,1 Blue Slip APT 14A,11222.0,40.735630,-73.959969,3620.0,0.65,Apartment,NaN,0 spaces,NaN,1.0,0.0,0.0,0.0,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,0.7,PK-5,True,Primary,Ps 31 Samuel F Dupont,8.0,594.0,15.0,Public,1.3,PK-8,True,Primary,Ps 84 Jose De Diego,5.0,795.0,10.0,Public,10647
10986,Brooklyn,NY,218 Frost St #2B,11211.0,40.718079,-73.942352,3690.0,0.65,Mon Feb 15 2021,Multiple Occupancy,"Cats, Small Dogs, Dogs",NaN,2.0,0.0,0.0,2.0,True,NaN,True,False,800 sqft,NaN,5.0,251.0,263153.0,0.3,PK-5,True,Primary,Ps 132 The Conselyea School,6.0,724.0,17.0,Public,0.6,6-8,True,Middle,Ms 582,7.0,260.0,12.0,Public,10986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73878,Brooklyn,NY,664 Humboldt St,11222.0,40.724380,-73.944954,10.0,0.65,Multiple Occupancy,1901,0 spaces,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,"2,538 sqft","1,800 sqft",2.0,4532.0,1788000.0,0.2,PK-5,True,Primary,Ps 110 The Monitor,4.0,522.0,14.0,Public,0.3,6-8,True,Middle,John Ericsson Middle School 126,4.0,377.0,11.0,Public,73878
73899,Brooklyn,NY,32 Diamond St,11222.0,40.723984,-73.945686,10.0,0.65,Multiple Occupancy,1901,0 spaces,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,"1,518 sqft","1,888 sqft",2.0,2595.0,1520000.0,0.2,PK-5,True,Primary,Ps 110 The Monitor,4.0,522.0,14.0,Public,0.3,6-8,True,Middle,John Ericsson Middle School 126,4.0,377.0,11.0,Public,73899
73926,Brooklyn,NY,117 Milton St,11222.0,40.729504,-73.956177,2150.0,0.65,Apartment,1874,On-street,NaN,1.0,1.0,0.0,0.0,False,False,False,False,290 sqft,"1,995 sqft",3.0,5392.0,2427000.0,0.4,PK-5,True,Primary,P.S. 34 Oliver H Perry,7.0,482.0,15.0,Public,0.8,6-8,True,Middle,John Ericsson Middle School 126,4.0,377.0,11.0,Public,73926
73937,Brooklyn,NY,217 N 11th St APT 3,11211.0,40.718796,-73.952965,10.0,0.65,Condo,2006,0 spaces,NaN,1.0,NaN,NaN,3.0,False,NaN,False,False,814 sqft,NaN,4.0,6101.0,315211.0,0.3,PK-5,True,Primary,Ps 17 Henry D Woodworth,5.0,267.0,10.0,Public,0.3,6-8,True,Middle,John Ericsson Middle School 126,4.0,377.0,11.0,Public,73937


In [44]:
brooklyn_df = brooklyn_df.drop(brooklyn_df.loc[brooklyn_df['price'] < 50000].index, axis=0)

In [45]:
brooklyn_df = brooklyn_df.drop(brooklyn_df.loc[brooklyn_df['price'] > 100000000].index, axis=0)

In [46]:
# We are going to simplify the Basement feature and make it into a boolean for all missing values we put false
brooklyn_df['Basement'] = brooklyn_df['Basement'].fillna(False)

In [47]:
#For all values that are none or unfinished we put false as well and make all the ones with basements True
for i in brooklyn_df.index:
    if type(brooklyn_df['Basement'][i]) == str:
        if brooklyn_df['Basement'][i] in ('None') or brooklyn_df['Basement'][i] in ('Unfinished') or brooklyn_df['Basement'][i] in ('None,Unfinished'):
            brooklyn_df['Basement'][i] = False
        else:
            brooklyn_df['Basement'][i] = True
            
        

<ipython-input-47-826dc5513c79>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brooklyn_df['Basement'][i] = True
<ipython-input-47-826dc5513c79>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brooklyn_df['Basement'][i] = False


In [48]:
brooklyn_df['Basement'].value_counts()

False    11451
True      2914
Name: Basement, dtype: int64

### 1.6.5.1 Fixing Datatypes of Columns<a id='1.6.5.1_Fixing_datatypes_of_columns'></a>


In [49]:
#Numeric Features I need to make sure each column data type is accurate
brooklyn_df['LivingArea_Space(SqFT)'].dtype
#This is type object I will change to string then remove the sqft and turn to int

dtype('O')

In [50]:
brooklyn_df['LivingArea_Space(SqFT)'] = brooklyn_df['LivingArea_Space(SqFT)'].replace(to_replace ='sqft', value = '', regex = True)
#Get rid of sqft as we denouce it was in SqFT in the column name

In [51]:
#Also need to get rid of all space and comma's
brooklyn_df['LivingArea_Space(SqFT)'] = brooklyn_df['LivingArea_Space(SqFT)'].replace(to_replace =' ', value = "", regex = True)


In [52]:
brooklyn_df['LivingArea_Space(SqFT)'] = brooklyn_df['LivingArea_Space(SqFT)'].replace(to_replace =',', value = '', regex = True)


In [53]:
brooklyn_df['LivingArea_Space(SqFT)'] = brooklyn_df['LivingArea_Space(SqFT)'].replace(to_replace ='nan', value = np.nan, regex = True)


In [54]:
brooklyn_df['LivingArea_Space(SqFT)'] = brooklyn_df['LivingArea_Space(SqFT)'].astype('O')

In [55]:
brooklyn_df['LivingArea_Space(SqFT)'].loc[brooklyn_df['LivingArea_Space(SqFT)'].notnull()] = brooklyn_df['LivingArea_Space(SqFT)'].loc[brooklyn_df['LivingArea_Space(SqFT)'].notnull()].astype(int)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [56]:
stat.mean(brooklyn_df['LivingArea_Space(SqFT)'].loc[brooklyn_df['LivingArea_Space(SqFT)'].notnull()])

3027.082433758587

In [57]:
#Lets fix columns that have NaN for boolean values to false

brooklyn_df['Fireplace_Available'].fillna(False, inplace = True)

In [58]:

missing = pd.concat([brooklyn_df.isnull().sum(), 100 * brooklyn_df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count', '%'])


,count,%
City,0,0.000000
State,0,0.000000
address/streetAddress,0,0.000000
latitude,0,0.000000
longitude,0,0.000000
Basement,0,0.000000
Fireplace_Available,0,0.000000
Spa_Available,0,0.000000
id,0,0.000000
propertyTaxRate,1,0.006961


Bedrooms will be replaced with their respective price range, same with Total_Bathrooms as well.
Since Price is our target feature we will drop any rows without a price. For storied Building we will make it 1 at worst we can assume they have at least one floor.

In [59]:
brooklyn_df = brooklyn_df.drop(brooklyn_df.loc[brooklyn_df['price'].isnull()].index, axis=0)

In [60]:
brooklyn_df =brooklyn_df.reset_index(drop=True)

In [61]:
brooklyn_df['id'] = brooklyn_df.index

In [62]:

bedroom_df = (brooklyn_df[['Bedrooms','price']].loc[brooklyn_df['Bedrooms'].isnull()]) 
  
    

In [63]:
brooklyn_df.loc[brooklyn_df['price'] > 50000000]

,City,State,address/streetAddress,ZipCode,latitude,longitude,price,propertyTaxRate,HomeType,YearBuilt,ParkingType,Basement,Total_Bathrooms,Full_Bathrooms,Half_Bathrooms,Bedrooms,Cooling_Available,Fireplace_Available,Heating_Available,Spa_Available,LivingArea_Space(SqFT),Lot_Size,Story_Building,Annual_Tax,Tax_Assessed_Value,schools/0/distance,schools/0/grades,schools/0/isAssigned,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/type,schools/1/distance,schools/1/grades,schools/1/isAssigned,schools/1/level,schools/1/name,schools/1/rating,schools/1/size,schools/1/studentsPerTeacher,schools/1/type,id
896,Brooklyn,NY,383 Gold St,11201.0,40.692825,-73.983200,100000000.0,0.65,Vacant Land,NaN,0 spaces,False,0.0,NaN,NaN,0.0,False,False,False,False,NaN,"2,655 sqft",NaN,23170.0,733000.0,0.3,PK-5,True,Primary,Ps 287 Bailey K Ashford,1.0,175.0,13.0,Public,0.8,K-8,True,Elementary,Ps 8 Robert Fulton,8.0,916.0,15.0,Public,896
918,Brooklyn,NY,67 Duffield St,11201.0,40.696484,-73.983559,60000000.0,0.65,Vacant Land,NaN,0 spaces,False,0.0,NaN,NaN,0.0,False,False,False,False,NaN,0.63 Acres,NaN,149846.0,3270000.0,0.2,PK-5,True,Primary,Ps 287 Bailey K Ashford,1.0,175.0,13.0,Public,0.6,K-8,True,Elementary,Ps 8 Robert Fulton,8.0,916.0,15.0,Public,918
1311,Brooklyn,NY,96 Dekalb Ave,11201.0,40.689644,-73.978989,62000000.0,0.65,Multiple Occupancy,1930,0 spaces,False,1.0,0.0,0.0,1.0,True,True,True,False,3600,"1,507 sqft",4.0,11025.0,2240000.0,0.5,PK-5,True,Primary,Ps 20 Clinton Hill,7.0,447.0,15.0,Public,0.8,PK-8,True,Primary,Ps 282 Park Slope,7.0,736.0,14.0,Public,1311
1813,Brooklyn,NY,1752 Shore Pkwy,11214.0,40.592159,-73.998123,73000000.0,0.65,Single Family,2013,0 spaces,False,NaN,NaN,NaN,NaN,False,False,False,False,259823,11.47 Acres,1.0,79719.0,98014000.0,0.5,K-5,True,Elementary,P.S. 101 the Verrazano,10.0,865.0,14.0,Public,0.8,6-12,True,Middle,Kingsborough Early College School,7.0,656.0,15.0,Public,1813
1821,Brooklyn,NY,41 Bay St,11231.0,40.588680,-73.997162,53500000.0,0.65,Vacant Land,NaN,0 spaces,False,0.0,NaN,NaN,0.0,False,False,False,False,NaN,1.25 Acres,NaN,77621.0,1637000.0,0.7,K-5,True,Elementary,P.S. 101 the Verrazano,10.0,865.0,14.0,Public,0.8,6-12,True,Middle,Kingsborough Early College School,7.0,656.0,15.0,Public,1821
6313,Brooklyn,NY,96 Dekalb Ave,11201.0,40.689644,-73.978989,62000000.0,0.65,Multiple Occupancy,1930,0 spaces,False,1.0,0.0,0.0,1.0,True,True,True,False,3600,"1,507 sqft",4.0,11025.0,2240000.0,0.5,PK-5,True,Primary,Ps 20 Clinton Hill,7.0,447.0,15.0,Public,0.8,PK-8,True,Primary,Ps 282 Park Slope,7.0,736.0,14.0,Public,6313
7188,Brooklyn,NY,1320 37th St,11218.0,40.641647,-73.984367,53175570.0,0.65,Single Family,2016,0 spaces,False,0.0,NaN,NaN,0.0,False,False,False,False,124954,0.77 Acres,5.0,58063.0,16888000.0,0.3,PK-5,True,Primary,Ps 230 Doris L Cohen,6.0,1168.0,13.0,Public,0.8,6-8,True,Middle,Middle 839,5.0,326.0,NaN,Public,7188
12778,Brooklyn,NY,1712 E 14th St,11229.0,40.607674,-73.959496,76000000.0,0.65,Single Family,2018,"Garage, Garage - Attached, Covered",False,0.0,NaN,NaN,0.0,False,False,False,False,114433,0.51 Acres,5.0,122221.0,19204000.0,0.3,PK-5,True,Primary,Ps 255 Barbara Reing School,9.0,931.0,12.0,Public,0.3,6-8,True,Middle,Jhs 234 Arthur W Cunningham,8.0,1909.0,18.0,Public,12778
13808,Brooklyn,NY,206 Kent Ave,11249.0,40.717953,-73.964668,77285000.0,0.65,Single Family,2018,0 spaces,False,0.0,NaN,NaN,0.0,False,False,False,False,81172,0.52 Acres,6.0,64672.0,2250000.0,0.5,PK-5,True,Primary,Ps 17 Henry D Woodworth,5.0,267.0,10.0,Public,0.2,PK-8,True,Primary,Ps 84 Jose De Diego,5.0,795.0,10.0,Public,13808


In [64]:
for i in bedroom_df.index:
    if bedroom_df['price'][i] < 100000:
        bedroom_df['Bedrooms'][i] = round(brooklyn_df['Bedrooms'].loc[brooklyn_df['price'] < 100000].mean())
    if bedroom_df['price'][i] >= 100000 and bedroom_df['price'][i] < 500000:
        bedroom_df['Bedrooms'][i] = round(brooklyn_df['Bedrooms'].loc[(brooklyn_df['price'] >= 100000) & (brooklyn_df['price'][i] < 500000)].mean())
    if bedroom_df['price'][i] >= 500000 and bedroom_df['price'][i] < 1000000:
        bedroom_df['Bedrooms'][i] = round(brooklyn_df['Bedrooms'].loc[(brooklyn_df['price'] >= 500000) & (brooklyn_df['price'][i] < 1000000)].mean())
    if bedroom_df['price'][i] >= 1000000 and bedroom_df['price'][i] < 5000000:
        bedroom_df['Bedrooms'][i] = round(brooklyn_df['Bedrooms'].loc[(brooklyn_df['price'] >= 1000000) & (brooklyn_df['price'][i] < 5000000)].mean())
    if bedroom_df['price'][i] >= 5000000 and bedroom_df['price'][i] < 10000000:
        bedroom_df['Bedrooms'][i] = round(brooklyn_df['Bedrooms'].loc[(brooklyn_df['price'] >= 5000000) & (brooklyn_df['price'][i] < 10000000)].mean())
    if bedroom_df['price'][i] >= 10000000 and bedroom_df['price'][i] < 50000000:
        bedroom_df['Bedrooms'][i] = 6
    if bedroom_df['price'][i] >= 50000000:
        bedroom_df['Bedrooms'][i] = 6
        

In [65]:
brooklyn_df['Bedrooms'].loc[brooklyn_df['Bedrooms'].isnull()] = bedroom_df['Bedrooms']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [68]:
brooklyn_df.loc[brooklyn_df['Full_Bathrooms'].isnull()]

,City,State,address/streetAddress,ZipCode,latitude,longitude,price,propertyTaxRate,HomeType,YearBuilt,ParkingType,Basement,Total_Bathrooms,Full_Bathrooms,Half_Bathrooms,Bedrooms,Cooling_Available,Fireplace_Available,Heating_Available,Spa_Available,LivingArea_Space(SqFT),Lot_Size,Story_Building,Annual_Tax,Tax_Assessed_Value,schools/0/distance,schools/0/grades,schools/0/isAssigned,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/type,schools/1/distance,schools/1/grades,schools/1/isAssigned,schools/1/level,schools/1/name,schools/1/rating,schools/1/size,schools/1/studentsPerTeacher,schools/1/type,id
7,Brooklyn,NY,110 Van Dyke St,11231.0,40.676014,-74.014946,725000.0,0.65,NaN,NaN,NaN,False,NaN,NaN,NaN,4.0,NaN,False,NaN,False,NaN,"2,000 sqft",NaN,5280.0,NaN,0.2,PK-5,True,Primary,Ps 15 Patrick F Daly,2.0,487.0,9.0,Public,0.8,6-12,True,Middle,Brooklyn Secondary School For Collaborative St...,3.0,675.0,11.0,Public,7
8,Brooklyn,NY,383 Van Brunt St,11231.0,40.676037,-74.014069,3899999.0,0.65,Mixed Use,1925,"Street, Other",True,3.0,NaN,3.0,5.0,NaN,False,True,False,4092,"1,980 sqft",3.0,3342.0,366000.0,0.2,PK-5,True,Primary,Ps 15 Patrick F Daly,2.0,487.0,9.0,Public,0.8,6-12,True,Middle,Brooklyn Secondary School For Collaborative St...,3.0,675.0,11.0,Public,8
10,Brooklyn,NY,178 Coffey St,11231.0,40.677788,-74.015991,2200000.0,0.65,Multiple Occupancy,1899,0 spaces,False,NaN,NaN,NaN,5.0,False,False,False,False,2884,"2,000 sqft",2.0,2888.0,2682000.0,0.3,PK-5,True,Primary,Ps 15 Patrick F Daly,2.0,487.0,9.0,Public,0.9,6-12,True,Middle,Brooklyn Secondary School For Collaborative St...,3.0,675.0,11.0,Public,10
11,Brooklyn,NY,176 Coffey St,11231.0,40.677753,-74.015938,2010000.0,0.65,Multiple Occupancy,1899,0 spaces,False,NaN,NaN,NaN,5.0,False,False,False,False,2884,"2,000 sqft",2.0,3881.0,2682000.0,0.3,PK-5,True,Primary,Ps 15 Patrick F Daly,2.0,487.0,9.0,Public,0.9,6-12,True,Middle,Brooklyn Secondary School For Collaborative St...,3.0,675.0,11.0,Public,11
17,Brooklyn,NY,153 Van Dyke St,11231.0,40.675964,-74.015572,3600000.0,0.65,Vacant Land,NaN,0 spaces,False,0.0,NaN,NaN,0.0,False,False,False,False,NaN,"2,000 sqft",NaN,6183.0,194000.0,0.2,PK-5,True,Primary,Ps 15 Patrick F Daly,2.0,487.0,9.0,Public,0.9,6-12,True,Middle,Brooklyn Secondary School For Collaborative St...,3.0,675.0,11.0,Public,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14347,Brooklyn,NY,65 N 7th St,11249.0,40.720154,-73.960999,4000000.0,0.65,Single Family,1930,0 spaces,False,0.0,NaN,NaN,0.0,False,False,False,False,3500,"2,500 sqft",2.0,47397.0,1195000.0,0.4,PK-5,True,Primary,Ps 17 Henry D Woodworth,5.0,267.0,10.0,Public,0.4,PK-8,True,Primary,Ps 84 Jose De Diego,5.0,795.0,10.0,Public,14347
14348,Brooklyn,NY,70 N 6th St,11249.0,40.719044,-73.961609,13766650.0,0.65,Single Family,1920,0 spaces,False,0.0,NaN,NaN,0.0,False,False,False,False,15000,"7,500 sqft",2.0,86232.0,2021000.0,0.4,PK-5,True,Primary,Ps 17 Henry D Woodworth,5.0,267.0,10.0,Public,0.3,PK-8,True,Primary,Ps 84 Jose De Diego,5.0,795.0,10.0,Public,14348
14349,Brooklyn,NY,159 Wythe Ave,11249.0,40.719036,-73.960686,8500000.0,0.65,Single Family,1910,0 spaces,False,0.0,NaN,NaN,0.0,False,False,False,False,8050,"2,980 sqft",4.0,17157.0,1169000.0,0.3,PK-5,True,Primary,Ps 17 Henry D Woodworth,5.0,267.0,10.0,Public,0.3,PK-8,True,Primary,Ps 84 Jose De Diego,5.0,795.0,10.0,Public,14349
14351,Brooklyn,NY,168 Wythe Ave,11249.0,40.719543,-73.960777,11711000.0,0.65,Single Family,1910,0 spaces,False,NaN,NaN,NaN,6.0,False,False,False,False,11750,"5,000 sqft",3.0,26659.0,4177200.0,0.3,PK-5,True,Primary,Ps 17 Henry D Woodworth,5.0,267.0,10.0,Public,0.3,PK-8,True,Primary,Ps 84 Jose De Diego,5.0,795.0,10.0,Public,14351


In [69]:

bathroom_df = (brooklyn_df[['Total_Bathrooms','price']].loc[brooklyn_df['Total_Bathrooms'].isnull()]) 
  
    

In [70]:
for i in bathroom_df.index:
    if bathroom_df['price'][i] < 100000:
        bathroom_df['Total_Bathrooms'][i] = round(brooklyn_df['Total_Bathrooms'].loc[brooklyn_df['price'] < 100000].mean())
    if bathroom_df['price'][i] >= 100000 and bathroom_df['price'][i] < 500000:
        bathroom_df['Total_Bathrooms'][i] = round(brooklyn_df['Total_Bathrooms'].loc[(brooklyn_df['price'] >= 100000) & (brooklyn_df['price'][i] < 500000)].mean())
    if bathroom_df['price'][i] >= 500000 and bathroom_df['price'][i] < 1000000:
        bathroom_df['Total_Bathrooms'][i] = round(brooklyn_df['Total_Bathrooms'].loc[(brooklyn_df['price'] >= 500000) & (brooklyn_df['price'][i] < 1000000)].mean())
    if bathroom_df['price'][i] >= 1000000 and bathroom_df['price'][i] < 5000000:
        bathroom_df['Total_Bathrooms'][i] = round(brooklyn_df['Total_Bathrooms'].loc[(brooklyn_df['price'] >= 1000000) & (brooklyn_df['price'][i] < 5000000)].mean())
    if bathroom_df['price'][i] >= 5000000 and bathroom_df['price'][i] < 10000000:
        bathroom_df['Total_Bathrooms'][i] = round(brooklyn_df['Total_Bathrooms'].loc[(brooklyn_df['price'] >= 5000000) & (brooklyn_df['price'][i] < 10000000)].mean())
    if bathroom_df['price'][i] >= 10000000 and bathroom_df['price'][i] < 50000000:
        bathroom_df['Total_Bathrooms'][i] = round(brooklyn_df['Total_Bathrooms'].loc[(brooklyn_df['price'] >= 10000000) & (brooklyn_df['price'][i] < 50000000)].mean())
    if bathroom_df['price'][i] >= 50000000:
        bathroom_df['Total_Bathrooms'][i] = round(brooklyn_df['Total_Bathrooms'].loc[(brooklyn_df['price'] >= 50000000)].mean())
        

In [71]:
brooklyn_df['Total_Bathrooms'].loc[brooklyn_df['Total_Bathrooms'].isnull()] = bathroom_df['Total_Bathrooms']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [72]:
#We are going to fill in NaN for story building column with 1 we can assume at worse they have at least one floor 
brooklyn_df['Story_Building'].fillna(1,inplace=True)

### 1.6.5.2 Reverse Geolocator<a id='1.6.5.2_Reverse_Geolocator'></a>

We are going to use a reverse geolocator using the library geopy to get the proper address, neighborhood, and borough using longitude and latitude.

In [73]:
# Lets see an example using our first observation 60 Terrace View Ave
def geolocator(coordinates):
    geolocator = Nominatim(user_agent="Pizon")
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)
    location = reverse(coordinates, language='en', exactly_one=True)
    return location.raw['address']


In [75]:
# .address will give us its address, Neighborhood, borough, county, state, and zip code
geolocator((40.892689,-73.910667))

{'house_number': '625',
 'road': 'West 246th St',
 'borough': 'The Bronx',
 'neighbourhood': 'Bronx County',
 'city': 'New York',
 'state': 'New York',
 'postcode': '10471',
 'country': 'United States',
 'country_code': 'us'}

In [77]:
brooklyn_df['latitude'][0:100]

0     40.725403
1     40.727341
2     40.678719
3     40.677280
4     40.675514
        ...    
95    40.714432
96    40.709633
97    40.726913
98    40.732056
99    40.714367
Name: latitude, Length: 100, dtype: float64

In [ ]:
#Now we need to make a function to get the address for each latitude and longitude and put in the right address
#Iterating through dataframe rows takes forever we going to make our latitude and longtitudes into numpy arrays
np_latitude = brooklyn_df['latitude'].to_numpy()
np_longitude = brooklyn_df['longitude'].to_numpy()
Neighborhood = []
np_index = brooklyn_df['id'].to_numpy()
    

In [ ]:
for i, j, k in zip(np_latitude, np_longitude, np_index):
    coordinates = (i,j)
    address = geolocator(coordinates)
    if 'neighbourhood' in address: 
        n = address['neighbourhood']
    elif 'town' in address:
        n = address['town']
    elif 'city_district' in address:
        n = address['city_district']
    else: n = None
    Neighborhood.append(n)

### 1.6.6 Distributions<a id='1.6.6_Distribution'></a>

### 1.6.6.1 Number of distinct neighborhoods<a id='1.6.4_Number_of_distinct_neighborhoods'></a>

In [ ]:
#Well that took forever we can see that there are 57 distinct neighborhoods while a lot of the neighborhoods are missing 
#due our api not able to properly provide a neighborhood, so we will count at that in the kings county district. 
#There are 57 distinct neighborhoods given that we seperate their Historic district 
Counter(Neighborhood).most_common()

In [ ]:
brooklyn_df['Neighborhood'] = Neighborhood

In [ ]:
#Lets re-order our columns to properly line up 
column_list = brooklyn_df.columns

In [ ]:
column_list = ['City', 'Neighborhood', 'State', 'address/streetAddress', 'ZipCode', 'latitude',
       'longitude', 'price', 'propertyTaxRate', 'HomeType', 'YearBuilt',
       'ParkingType', 'Basement', 'Total_Bathrooms', 'Full_Bathrooms',
       'Half_Bathrooms', 'Bedrooms', 'Cooling_Available',
       'Fireplace_Available', 'Heating_Available', 'Spa_Available',
       'LivingArea_Space(SqFT)', 'Lot_Size', 'Story_Building', 'Annual_Tax',
       'Tax_Assessed_Value', 'schools/0/distance', 'schools/0/grades',
       'schools/0/isAssigned', 'schools/0/level', 'schools/0/name',
       'schools/0/rating', 'schools/0/size', 'schools/0/studentsPerTeacher',
       'schools/0/type', 'schools/1/distance', 'schools/1/grades',
       'schools/1/isAssigned', 'schools/1/level', 'schools/1/name',
       'schools/1/rating', 'schools/1/size', 'schools/1/studentsPerTeacher',
       'schools/1/type', 'id']
brooklyn_df = brooklyn_df.reindex(columns = column_list)

### 1.6.6.2 Distribution Of Housing Price By Neighborhood<a id='1.6.6.2_Distribution_Of_Housing_Price_By_Neighborhood'></a>


In [ ]:
house_prices_neighborhood = brooklyn_df[['Neighborhood','price']].groupby('Neighborhood', as_index=False).mean().round()

In [ ]:
house_prices_neighborhood

In [ ]:
#This is a function by Dan Friedman to change the labels of large numbers 
#Here is a link
#https://dfrieds.com/data-visualizations/how-format-large-tick-values.html
sns.set(font_scale=1.4)

def reformat_large_tick_values(tick_val, pos):
    """
    Turns large tick values (in the billions, millions and thousands) such as 4500 into 4.5K and also appropriately turns 4000 into 4K (no zero after the decimal).
    """
    if tick_val >= 1000000000:
        val = round(tick_val/1000000000, 1)
        new_tick_format = '{:}B'.format(val)
    elif tick_val >= 1000000:
        val = round(tick_val/1000000, 1)
        new_tick_format = '{:}M'.format(val)
    elif tick_val >= 1000:
        val = round(tick_val/1000, 1)
        new_tick_format = '{:}K'.format(val)
    elif tick_val < 1000:
        new_tick_format = round(tick_val, 1)
    else:
        new_tick_format = tick_val

    # make new_tick_format into a string value
    new_tick_format = str(new_tick_format)
    
    # code below will keep 4.5M as is but change values such as 4.0M to 4M since that zero after the decimal isn't needed
    index_of_decimal = new_tick_format.find(".")
    
    if index_of_decimal != -1:
        value_after_decimal = new_tick_format[index_of_decimal+1]
        if value_after_decimal == "0":
            # remove the 0 after the decimal point since it's not needed
            new_tick_format = new_tick_format[0:index_of_decimal] + new_tick_format[index_of_decimal+2:]
            
    return new_tick_format


In [ ]:
#Lets bar plot the average price of houses by thier neighborhoods 
plt.subplots(figsize=(12, 8))
sns.barplot(x='Neighborhood', y='price', data=house_prices_neighborhood)
plt.xticks(rotation='vertical')
plt.ylabel('Average Price($)')
plt.xlabel('Neighborhoods')
plt.ticklabel_format(style='plain', axis='y')
ax = plt.gca()
ax.yaxis.set_major_formatter(tick.FuncFormatter(reformat_large_tick_values));


In [ ]:
#Downtown prices seems really high lets check it out
brooklyn_df[brooklyn_df['Neighborhood'] == 'Downtown Brooklyn']
#It seems it is being skewed by one row which is a large lot as a office space which being sold for 100mil so we should
#drop this oberservation.

In [ ]:
#We should drop all hometype which is vacant land as these are not typical houses and mess up the distribution 
#of house prices 
vacant_lands = brooklyn_df[brooklyn_df['HomeType'] == 'Vacant Land'].index
brooklyn_df = brooklyn_df.drop(vacant_lands)

In [ ]:
brooklyn_df = brooklyn_df.reset_index(drop=True)


In [ ]:
house_prices_neighborhood = brooklyn_df[['Neighborhood','price']].groupby('Neighborhood', as_index=False).mean().round()
plt.subplots(figsize=(12, 8))
sns.barplot(x='Neighborhood', y='price', data=house_prices_neighborhood)
plt.xticks(rotation='vertical')
plt.ylabel('Average Price($)')
plt.xlabel('Neighborhoods')
plt.ticklabel_format(style='plain', axis='y')
ax = plt.gca()
ax.yaxis.set_major_formatter(tick.FuncFormatter(reformat_large_tick_values));


## 1.7 Neighborhood Summary Statistics <a id='1.7_Neighborhood_Summary_Statistics'></a>

In [ ]:
neighborhood_summary = brooklyn_df.groupby('Neighborhood').agg(
    houses_in_neighborhood = pd.NamedAgg(column='Neighborhood', aggfunc='count'),
    average_price = pd.NamedAgg(column='price', aggfunc='mean'),
    average_bedrooms = pd.NamedAgg(column='Bedrooms', aggfunc='mean'),
    average_bathrooms = pd.NamedAgg(column='Total_Bathrooms', aggfunc='mean'),
    average_stories_of_buidling = pd.NamedAgg(column='Story_Building', aggfunc='mean')
)

In [ ]:
neighborhood_summary['average_price'] = round(neighborhood_summary['average_price'])
neighborhood_summary['average_bedrooms'] = round(neighborhood_summary['average_bedrooms'])
neighborhood_summary['average_bathrooms'] = round(neighborhood_summary['average_bathrooms'])
neighborhood_summary['average_stories_of_buidling'] = round(neighborhood_summary['average_stories_of_buidling'])

In [ ]:
neighborhood_summary

## 1.8 Save Data <a id='1.8_Save_Data'></a>

In [ ]:
#Lets see what our new dataframe looks like
brooklyn_df.info()
brooklyn_df.shape

In [ ]:
brooklyn_df.to_csv(r'C:\Users\izine\Desktop\GitHub\Springboard-Data-Science\Capstone 1\Data\brooklyn_df_cleaned.csv', index=False)

## 1.9 Summary <a id='1.9_Summary'></a>

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(brooklyn_df, title='Pandas Profiling Report', explorative=True)


In [ ]:
profile

We are tasked to best estimates prices for houses being built by Capital Fortune real-estate company, who are building houses in Brooklyn, NY. We extracted data from the nyc_housing data from zillow. We need to build a predict model which can best estimate prices for the houses given the brooklyn market. 

We started by inspecting our original dataframe which had over 75k rows and 1507 columns, we began to start cleaning our data as a lot of our columns which were not relevant or repeated information of other columns. We needed to isolate the dataframe for our target location which is Brooklyn so we filtered our nyc dataframe to all address which resided in Brooklyn. 

Next step was to clean up some of the data types for our columns, unfortunately there were a lot of missing data to counter-act we either dropped those columns or imputed data to some important columns. For instance we change our column type of Has_Basement to True or False to simplify this feature. our column names were quite un-intuitive from the original datafile so we relabeled a lot of them for more readable names.  

We noticed there were a lot fo houses which were sold for prices far below their market value for instance a million dollar house was sold for 5000, these are cases where family members sell their property to other family and friends to change ownership and etc. So we excluded these houses, the opposite was also the case there were few observations where their price was 100 million and above we realized a lot of the hometype labeled vacant land were lots for large office space and warehouses behind sold, as these properties does not relate to houses being built by Capital Fortune we ommited these as well. 

We wanted to do an analysis on neighborhoods in brooklyn and how their prices differ so we utilized a geolocator to reverse geocode neighborhoods using latitudes and longitudes which was in our dataframe. One issue was a lot of neighborhood in the Kings County distinct was not coming up in our API so we left them as Kings County thus why they have the largest number of rows. 

Reverse geolocation was a interesting idea but the time it takes to process is unbearable, as we are capped to one requests per second with our API, it took hours to finish our neighborhood look up. But once we gathered all the neighborhoods via its latitude and longitude we were able to see how the average price of their houses were distributed for each neighborhood. We also peeked into some neighborhood summary

While this was a cleaning of the data I believe we better fine tune our data and perphaps impute more missing data based on better classification rather than just using the average. 